In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline  
import numpy as np
import pickle as plk
import pwlf
import os

import plotly.express as px

In [24]:
# Mimic drop down
building_dropdown = 'CD Howe'
year_dropdown = 2017

In [25]:
# Import Data
PATH = pathlib.Path().parent
DATA_PATH = PATH.joinpath("../data").resolve()

with open(DATA_PATH.joinpath(f'{building_dropdown}/Energy/_energyData.pkl'), 'rb') as f:
    energy_df = plk.load(f)

with open(DATA_PATH.joinpath(f'Supporting/Weather/Ottawa.pkl'), 'rb') as f:
    weather_df = plk.load(f)
    
df = pd.merge(energy_df, weather_df, how='outer', left_index=True, right_index=True)
df = df[df.index.year == year_dropdown]

df

,chilledWater,steam,electricity,temperature,relative_humidity,wind_speed
ts,,,,,,
2017-01-01 00:00:00,1598.09,10149.79,13061.04,-8.4,91.0,6.0
2017-01-01 01:00:00,1553.05,9949.16,13082.51,-8.0,91.0,6.0
2017-01-01 02:00:00,1639.66,8111.33,12937.88,-7.5,91.0,5.0
2017-01-01 03:00:00,1429.29,10358.87,12938.53,-6.9,92.0,6.0
2017-01-01 04:00:00,1543.53,9531.55,12954.43,-6.2,92.0,4.0
...,...,...,...,...,...,...
2017-12-31 19:00:00,1173.15,16949.53,12309.58,-21.5,61.0,13.0
2017-12-31 20:00:00,1115.01,18631.80,12200.93,-21.2,63.0,18.0
2017-12-31 21:00:00,1082.79,18089.81,12187.28,-22.3,66.0,10.0


In [26]:
steam_line=[[5, 17], np.array([-26.8      ,   2.7013179,  14.8465908,  32.3      ]), np.array([23014.69570784,  8437.05520873,  1101.6920742 ,  -557.85267625]), np.array([-26.3       ,   1.79999962,  11.9995312 ,  30.6       ]), np.array([18008.10193165,  7037.52934693,  1342.43233668,  -897.59210098])]

In [43]:
def seperate_data_by_Schedule(Data, workDayStart, workDayEnd, WeekendSplit):
    if WeekendSplit:
        DataWeekend = Data[Data.index.weekday > 4]
        DataWeekday = Data[Data.index.weekday < 5]
        DataEvening = DataWeekday[(DataWeekday.index.hour <= workDayStart) | (DataWeekday.index.hour > workDayEnd)]
        DataOffSchedule = pd.concat([DataWeekend, DataEvening])
        DataOnSchedule = DataWeekday[(DataWeekday.index.hour > workDayStart) & (DataWeekday.index.hour <= workDayEnd)]

    else:
        DataOffSchedule = Data[(Data.index.hour <= workDayStart) | (Data.index.hour > workDayEnd)]
        DataOnSchedule = Data[(Data.index.hour > workDayStart) & (Data.index.hour <= workDayEnd)]

    return DataOnSchedule, DataOffSchedule


def schedule_seperate_graph(data, sch, parameter, color_on, color_off, fit_line, color_line_on, color_line_off, weekend_included=True):
    """

    :param data: Dataframe with combined energy data and
    :param sch: Start and end of occupied period as dictionary [start, end]
    :param parameter: steam/chilledWater/electrical
    :param color_on:
    :param color_off:
    :return: Figure with coloured
    """

    DataOnSchedule, DataOffSchedule = seperate_data_by_Schedule(data, sch[0], sch[1], weekend_included)

    fig = go.Figure()
    fig.add_scatter(name='Off Schedule', x=DataOffSchedule.temperature, y=DataOffSchedule[parameter],
                    customdata=DataOffSchedule.index, mode='markers', marker=dict(color=color_off))
    fig.add_scatter(name='On Schedule', x=DataOnSchedule.temperature, y=DataOnSchedule[parameter],
                    customdata=DataOnSchedule.index, mode='markers', marker=dict(color=color_on))
    
    if fit_line != []:
        if fit_line[0] == sch:
    
            fig.add_scatter(name='_nolegend_', x=fit_line[1], y=fit_line[2], marker=dict(color=color_line_on), showlegend=False)  
            fig.add_scatter(name='_nolegend_', x=fit_line[3], y=fit_line[4], marker=dict(color=color_line_off), showlegend=False)   
    
    
    fig.update_layout(template="none", xaxis_title="Outdoor Temperature", yaxis_title="Energy (GJ/hr)")
    fig.update_layout(legend=dict(
        yanchor="top",
        y=-0.1,
        xanchor="left",
        x=0.99
    ))
    fig.update_traces(hovertemplate='Energy: %{y}<br>Date: %{customdata}')
    return fig

schedule_seperate_graph(df, [5, 17], 'steam', 'red', 'LightCoral', steam_line, 'black', 'grey', weekend_included=False)

In [6]:
def LinModel(Data, sch, parameter, WeekendSplit=True):

    # clean data for
    linData = Data.copy()
    linData = linData.dropna(subset=['temperature', parameter])
    linData = linData[linData[parameter] != 0]

    linDataOnSchedule, linDataOffSchedule = seperate_data_by_Schedule(linData, sch[0], sch[1], WeekendSplit)

    ## Find Change Point Models
    pwlf_On = pwlf.PiecewiseLinFit(linDataOnSchedule['temperature'].to_numpy(), linDataOnSchedule[parameter].to_numpy())
    cpts_On = pwlf_On.fit(3)
    pwlf_Off = pwlf.PiecewiseLinFit(linDataOffSchedule['temperature'].to_numpy(),
                                    linDataOffSchedule[parameter].to_numpy())
    cpts_Off = pwlf_Off.fit(3)

    return pwlf_On, pwlf_Off

pwlf_On, pwlf_Off = LinModel(df, [6, 18], 'electricity')

C:\Users\conco\anaconda3\lib\site-packages\pwlf\pwlf.py:1108: RuntimeWarning:

invalid value encountered in double_scalars



In [36]:
with open(DATA_PATH.joinpath(f'Supporting/Weather/Ottawa.pkl'), 'rb') as f:
    weather_df = plk.load(f)
df_weather = weather_df['temperature']

In [52]:
df_weather = df_weather[df_weather.index.year == 2019].fillna(method='ffill')
data = df_weather.to_json(date_format='iso', orient='split')
data = pd.read_json(data, typ='series', orient='split')
data

2019-01-01 00:00:00+00:00   -1.7
2019-01-01 01:00:00+00:00   -0.5
2019-01-01 02:00:00+00:00    0.1
2019-01-01 03:00:00+00:00    0.4
2019-01-01 04:00:00+00:00    0.1
                            ... 
2019-12-31 19:00:00+00:00    0.4
2019-12-31 20:00:00+00:00    0.2
2019-12-31 21:00:00+00:00   -0.3
2019-12-31 22:00:00+00:00   -0.6
2019-12-31 23:00:00+00:00   -0.8
Name: temperature, Length: 8760, dtype: float64

In [59]:
no_sch = pwlf_On.predict(data).sum()
d = {'No Schedule': [no_sch], 'bruh': [no_sch], 'haters': [no_sch]}
result_table = pd.DataFrame(data=d)
result_table.T

,0
No Schedule,1.301111e+08
bruh,1.301111e+08
haters,1.301111e+08


In [71]:
fig = px.bar(result_table.T)
fig.update_layout(template="simple_white", yaxis_title="Energy (GJ)", showlegend=False, xaxis_title=None)
fig

In [27]:
df = pd.read_csv('Ottawa_TMY.csv')
df.ts = pd.to_datetime(df.ts)
df = df.set_index('ts')

with open('Ottawa_TMY.pkl', 'wb') as f:
    plk.dump(df, f)

df

,temperature
ts,
2020-01-01 00:00:00,-16.0
2020-01-01 01:00:00,-16.0
2020-01-01 02:00:00,-16.0
2020-01-01 03:00:00,-17.0
2020-01-01 04:00:00,-17.0
...,...
2020-12-31 19:00:00,-17.0
2020-12-31 20:00:00,-16.0
2020-12-31 21:00:00,-16.0


In [28]:
with open(f'Ottawa_TMY.pkl', 'rb') as f:
    df2 = plk.load(f)
    
df2

,temperature
ts,
2020-01-01 00:00:00,-16.0
2020-01-01 01:00:00,-16.0
2020-01-01 02:00:00,-16.0
2020-01-01 03:00:00,-17.0
2020-01-01 04:00:00,-17.0
...,...
2020-12-31 19:00:00,-17.0
2020-12-31 20:00:00,-16.0
2020-12-31 21:00:00,-16.0
